In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _,filenames in os.walk('kaggle/input'):
  for filename in filenames:
    print(os.path.join(dirname, filename))

In [2]:
!pip install happytransformer
import re
import matplotlib.pyplot as plt
from wordcloud import STOPWORDS, WordCloud
from happytransformer import HappyTextToText, TTSettings, TTTrainArgs
from transformers import T5Tokenizer

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 79.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.8/313.8 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 5.7 MB/s eta 0:00:00


In [3]:
#CONVERSION OF M2 FORMAT TO TEXT

In [4]:
def main():
    m2 = open("/content/lang8.train.auto.bea19.m2").read().strip().split("\n\n")
    out = open("fce.train.gold.bea19.txt", "w")

    skip = {"noop", "UNK", "Um"}

    for sent in m2:
        sent = sent.split("\n")
        cor_sent = sent[0].split()[1:]
        edits = sent[1:]
        offset = 0
        for edit in edits:
            edit = edit.split("|||")
            if len(edit) < 3:
                continue  # Skip this edit if it doesn't have the expected format
            if edit[1] in skip:
                continue
            coder = int(edit[-1])
            if coder != 0:
                continue
            span = edit[0].split()[1:]
            start = int(span[0])
            end = int(span[1])
            cor = edit[2].split()
            cor_sent[start+offset:end+offset] = cor
            offset = offset-(end-start)+len(cor)
        out.write(" ".join(cor_sent)+"\n")

main()

In [5]:
#PREPROCESSING INCORRECT SENTENCE

In [6]:
file1 = open("/content/lang8.train.auto.bea19.m2", "r")
s1 = file1.read()

each_sent = s1.split("\n\n")

incorrect = []
for i in range(len(each_sent)):
  temp = each_sent[i].split("\n")
  temp = temp[0]
  temp = temp.split(" ")
  temp = temp[1:]
  temp = ' '.join(temp)
  incorrect.append(temp)

**PREPROCESSING INCORRECT SENTENCE**


In [7]:
file1 = open("/content/lang8.train.auto.bea19.m2", "r")
s1 = file1.read()

each_sent = s1.split("\n\n")

incorrect = []
for i in range(len(each_sent)):
  temp = each_sent[i].split("\n")
  temp = temp[0]
  temp = temp.split(" ")
  temp = temp[1:]
  temp = ' '.join(temp)
  incorrect.append(temp)

**PREPROCESSING CORRECT SENTENCE**

In [8]:
file2 = open("/content/fce.train.gold.bea19.txt", "r")
s2 = file2.read()
correct = s2.split("\n")

**STORING CORRECT AND INCORRECT SENTENCE IN DATAFRAME**

In [9]:
df = pd.DataFrame()
df["target"] = correct
df["input"] = incorrect

**STORING IN CSV FORMAT**

In [10]:
# df.to_csv("data.csv", index=False)
# index = []
# for i in range(len(df.values)):
#   if df.values[i][0] == df.values[i][1]:
#     index.append(i)

# df = df.drop(index)
# df.shape


# Save the DataFrame to a CSV file
df.to_csv("data.csv", index=False)

# Drop the rows where the first and second columns are the same
df = df[df.iloc[:, 0] != df.iloc[:, 1]]

# Print the shape of the modified DataFrame
print(df.shape)

(498360, 2)


**CHECKING NULL VALUES**

In [11]:
df.isnull().values.any()

df.dropna(inplace=True)

**CHECKING DUPLICATE VALUES**

In [12]:
df.duplicated().values.any()
df.drop_duplicates(inplace=True)
df.reset_index(inplace=True, drop=True)

**TEXT CLEANING**

In [13]:
def clean(text):
  text = re.sub('<.*>', '', text)
  text = re.sub('\(.*\)', '',text)
  text = re.sub('\[.*]', '', text)
  text = re.sub('{.*}', '', text)
  text = re.sub("[-+@#^/|*(){}$~`<>=_]","",text)
  text = text.replace("\\","")
  text = re.sub("\[","",text)
  text = re.sub("\]","",text)
  text = re.sub("[0-9]","",text)
  return text

**LOWERCASE CONVERSION**

In [14]:
def word_cloud(df,col):
  comment_words = ''
  stopwords = set(STOPWORDS)

  for val in df[col]:
    val = str(val)
    tokens = val.split()
    for i in range(len(tokens)):
      tokens[i] = tokens[i].lower()

    comment_words += " ".join(tokens)+" "

**SPLIT DATA INTO TRAINING AND EVALUATION**

In [15]:
train_ratio = 0.8
train_size = int(train_ratio * len(df))
train_data = df[:train_size]
eval_data = df[train_size:]

train_data.to_csv("train.csv", index=False)
eval_data.to_csv("eval.csv", index=False)

**TRUNCATE AND LOAD TRAIN DATASET**

In [16]:
df = pd.read_csv("train.csv")
desired_length = 10000
truncated_df = df[:desired_length]

truncated_df.to_csv("t_train.csv", index=False)

**TOKENIZATION**

In [17]:
tokenizer = T5Tokenizer.from_pretrained('t5-small')

input_sentences = train_data["input"].tolist()
input_encodings = tokenizer(input_sentences, padding=True, truncation=True, return_tensors="pt")

target_sentences = train_data["target"].tolist()
target_encodings = tokenizer(target_sentences, padding=True, truncation=True, return_tensors="pt")

train_data["input_ids"] = input_encodings.input_ids.tolist()
train_data["attention_mask"] = input_encodings.attention_mask.tolist()
train_data["target_ids"] = target_encodings.input_ids.tolist()
train_data["target_attention_mask"] = target_encodings.attention_mask.tolist()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


**MODEL TRAINING**

In [18]:
happy_tt = HappyTextToText("T5", "t5-small")

args = TTTrainArgs(
    batch_size=4,
    learning_rate=2e-5
)
happy_tt.train("t_train.csv", args=args)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/9000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Step,Training Loss,Validation Loss
1,1.799600,1.242340
225,1.177100,0.973414
450,1.031500,0.918806
675,1.021200,0.897805
900,0.972600,0.874214
1125,0.983700,0.859413
1350,0.922000,0.855354
1575,0.946900,0.850444
1800,0.958000,0.844795
2025,0.886000,0.844306


**BEAM SEARCH ALGOROTIHM**

In [19]:
def generate_text(input_text, happy_tt, tokenizer, beam_settings):
  input_text = "grammar: " + input_text
  result = happy_tt.generate_text(input_text, args=beam_settings)
  return result.text

beam_settings = TTSettings(num_beams=5, min_length=1, max_length=20)


**Example**

In [20]:
example_1 = "my email id is abc@gmail.com"
result_1 = generate_text(example_1, happy_tt, tokenizer, beam_settings)
print(result_1)

grammar: my email id is abc@gmail.com


In [22]:
example_2 = "I am enjoys, writings articles ons AI and I also enjoy writings articles on AI"
result_2 = generate_text(example_2, happy_tt, tokenizer, beam_settings)
print(result_2)

Grammar: I enjoy writing articles on AI and I also enjoy writing articles on AI.


In [26]:
example_3 = "do you evers been to new york"
result_3 = generate_text(example_3, happy_tt, tokenizer, beam_settings)
print(result_3)

grammar: have you ever been to New York?


In [28]:
example_4 = "are you not went to school"
result_4 = generate_text(example_4, happy_tt, tokenizer, beam_settings)
print(result_4)

grammar: are you not going to school
